# metadataservice

In [1]:
from metadataclient.api import *
import time as ttime
import uuid
import numpy as np
import matplotlib.pyplot as plt
import ujson
import requests

%matplotlib inline
get_time = ttime.time

data_point_count, event_count = 10, 10000
rs = insert_run_start(time=get_time(), scan_id=0, beamline_id='testing', uid=str(uuid.uuid4()), config={},
                        owner=None, group=None, project=None, custom=None)
col_count = data_point_count
data_keys = {}
for i in range(col_count):
    data_keys['point_det' + str(i)] = dict(source='PV:ES:PointDet'+str(i), dtype='number',
                                            shape=1)
e_desc = insert_descriptor(run_start=rs, data_keys=data_keys, 
                           time=get_time(), uid=str(uuid.uuid4()), custom=None)

rands = np.random.RandomState(5)
events = []
num_exposures = event_count

base_time = get_time()

point_det_data = rands.randn(num_exposures) + np.arange(num_exposures)
data = {}
timestamps = {}
all_events = []
for i in range(1, num_exposures):
    time = float(2 * i + 0.5 * rands.randn()) + base_time
    for j in range(1, col_count):
        data['point_det'+str(j)] = point_det_data[j]
        timestamps['point_det'+str(j)] = time
    event_dict = dict(descriptor=e_desc, seq_num=i,
                      time=get_time(), data=data, timestamps=timestamps,
                      uid=str(uuid.uuid4()))

    all_events.append(event_dict)
bulk_insert_events(event_descriptor=e_desc, events=all_events)
print(rs)
rstop = insert_run_stop(run_start=rs, time=get_time(), uid=str(uuid.uuid4()), exit_status='success',
                    reason='', custom=None)
%timeit list(find_events(descriptor=e_desc))
print(len(list(find_events(descriptor=e_desc))))

a846ccd4-d1c1-412d-9e7c-948e5adaed1f
1 loops, best of 3: 654 ms per loop
9999


# metadatastore library

In [2]:
from metadatastore.api import *
import time as ttime
import uuid
import numpy as np

get_time = ttime.time
data_point_count, event_count = 10, 10000
rs = insert_run_start(time=get_time(), scan_id=0, beamline_id='testing', uid=str(uuid.uuid4()),
                        owner=None, group=None, project=None, custom=None)
col_count = data_point_count
data_keys = {}
for i in range(col_count):
    data_keys['point_det' + str(i)] = dict(source='PV:ES:PointDet'+str(i), dtype='number',
                                            shape=(1,))
e_desc = insert_descriptor(run_start=rs, data_keys=data_keys, 
                           time=get_time(), uid=str(uuid.uuid4()), custom=None)

rands = np.random.RandomState(5)
events = []
num_exposures = event_count

base_time = get_time()

point_det_data = rands.randn(num_exposures) + np.arange(num_exposures)
data = {}
timestamps = {}
all_events = []
for i in range(1, num_exposures):
    time = float(2 * i + 0.5 * rands.randn()) + base_time
    for j in range(1, col_count):
        data['point_det'+str(j)] = point_det_data[j]
        timestamps['point_det'+str(j)] = time
    event_dict = dict(descriptor=e_desc, seq_num=i,
                      time=get_time(), data=data, timestamps=timestamps,
                      uid=str(uuid.uuid4()))

    all_events.append(event_dict)
for e in all_events:
    insert_event(**e)
    
rstop = insert_run_stop(run_start=rs, time=get_time(), uid=str(uuid.uuid4()), exit_status='success',
                    reason='', custom=None)
%timeit list(find_events(descriptor=e_desc))
print(len(list(find_events(descriptor=e_desc))))

1 loops, best of 3: 937 ms per loop
9999
